### HybridSPEA2GWOABC

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import warnings
from sklearn.metrics import confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import random

# Suppress warnings
warnings.filterwarnings('ignore')



def feature_evaluation(selected_columns, data, target, complexity_weight=1):
    X = data[selected_columns]
    y = target
    clf = GaussianNB()   # Using Random Forest classifier

    # Use StratifiedKFold for cross-validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(clf, X, y, cv=skf)

    accuracy = np.mean(cv_scores)
    robustness = np.std(cv_scores)

    num_features = len(selected_columns)
    complexity = complexity_weight / num_features if num_features > 0 else float('inf')
    interpretability = complexity
    return (accuracy, robustness, complexity, interpretability)


def feature_evaluation_with_complexity_weight(selected_columns, data, target, complexity_weight):
    # Adjusts feature evaluation to give more importance to complexity
    accuracy, robustness, complexity, interpretability = feature_evaluation(selected_columns, data, target)
    weighted_complexity_score = complexity_weight * complexity
    return (accuracy, -robustness, weighted_complexity_score, interpretability)

# Fitness Function
def _fitness(individual, data, target, complexity_weight=0.3):
    selected_features = data.columns[individual == 1]
    return feature_evaluation(selected_features, data, target, complexity_weight)

def update_archive(archive, population, fitness, archive_size):
    # Combine the current archive and the new population
    combined = np.vstack((archive, population))
    combined_fitness = np.hstack((spea2_fitness_assignment(archive, data, target),
                                  spea2_fitness_assignment(population, data, target)))

    # Sort the combined array based on the fitness values
    sorted_indices = np.argsort(combined_fitness)
    updated_archive = combined[sorted_indices]

    # Keep only the top individuals up to the specified archive size
    if len(updated_archive) > archive_size:
        updated_archive = updated_archive[:archive_size]

    return updated_archive



# Grey Wolf Optimizer (GWO) Phase
def gwo_feature_evaluation(population, data, target, current_iter, max_iter):
    alpha, beta, delta = _find_leaders(population, data, target)
    a = 2 - 2 * (current_iter / max_iter)  

    for i in range(len(population)):
        wolf = population[i]
        for j in range(len(wolf)):
            r1, r2, r3 = np.random.rand(3)
            A1 = 2 * a * r1 - a
            C1 = 2 * r2
            D_alpha = abs(C1 * alpha[j] - wolf[j])
            X1 = alpha[j] - A1 * D_alpha

            A2 = 2 * a * r3 - a
            C2 = 2 * r2
            D_beta = abs(C2 * beta[j] - wolf[j])
            X2 = beta[j] - A2 * D_beta

            A3 = 2 * a * r1 - a
            C3 = 2 * r2
            D_delta = abs(C3 * delta[j] - wolf[j])
            X3 = delta[j] - A3 * D_delta

            new_position = (X1 + X2 + X3) / 3
            population[i][j] = 1 if new_position > 0.5 else 0

# Find Leaders for GWO
def _find_leaders(population, data, target):
    fitness_values = np.array([_fitness(individual, data, target) for individual in population])
    sorted_indices = np.argsort(fitness_values[:, 0])[::-1]  # Sort based on accuracy
    alpha = population[sorted_indices[0]]
    beta = population[sorted_indices[1]]
    delta = population[sorted_indices[2]]
    return alpha, beta, delta

# SPEA2 Fitness Assignment
def spea2_fitness_assignment(population, data, target):
    fitness_values = np.array([feature_evaluation(data.columns[individual == 1], data, target) for individual in population])
    domination_counts = np.zeros(len(fitness_values))
    dominated_solutions = [[] for _ in range(len(fitness_values))]
    strength = np.zeros(len(fitness_values))
    raw_fitness = np.zeros(len(fitness_values))

    for i in range(len(fitness_values)):
        for j in range(len(fitness_values)):
            if dominates(fitness_values[i], fitness_values[j]):
                dominated_solutions[i].append(j)
            elif dominates(fitness_values[j], fitness_values[i]):
                domination_counts[i] += 1

        strength[i] = len(dominated_solutions[i])
        raw_fitness[i] = sum(strength[j] for j in dominated_solutions[i])

    density = crowding_distance(fitness_values, calculate_non_dominated_ranks(fitness_values))
    fitness = raw_fitness + 1.0 / (2.0 + density)

    return fitness

# SPEA2 Environmental Selection
def spea2_environmental_selection(population, fitness, archive_size):
    sorted_indices = np.argsort(fitness)
    return population[sorted_indices][:archive_size]

# Artificial Bee Colony (ABC) Phase
def abc_feature_selection(archive, data, target):
    for i in range(len(archive)):
        solution = archive[i]

        # Employed and Onlooker Bees Phases
        for phase in ['employed', 'onlooker']:
            if phase == 'onlooker':
                # Calculate probabilities for onlooker bees
                fitness_values = [feature_evaluation(data.columns[sol == 1], data, target)[0] for sol in archive]
                prob = [f / sum(fitness_values) for f in fitness_values]
                if np.random.rand() >= prob[i]:
                    continue

            phi = np.random.uniform(-1, 1, size=len(solution))
            k = np.random.choice(len(archive))
            while k == i:
                k = np.random.choice(len(archive))
            v = solution + phi * (solution - archive[k])

            if feature_evaluation(data.columns[v == 1], data, target) > feature_evaluation(data.columns[solution == 1], data, target):
                archive[i] = v

        # Scout Bees Phase
        if np.random.rand() < 0.1:  # Scout bee probability
            archive[i] = np.random.randint(0, 2, len(solution))


# Additional SPEA2 functions: Dominates, Non-Dominated Ranks, Crowding Distance
def dominates(row, candidate_row):
    return all(r >= c for r, c in zip(row, candidate_row)) and any(r > c for r, c in zip(row, candidate_row))

def calculate_non_dominated_ranks(fitness_values):
    domination_counts = np.zeros(len(fitness_values))
    dominated_solutions = [[] for _ in range(len(fitness_values))]
    ranks = np.zeros(len(fitness_values))

    for i, row in enumerate(fitness_values):
        for j, candidate_row in enumerate(fitness_values):
            if dominates(row, candidate_row):
                dominated_solutions[i].append(j)
            elif dominates(candidate_row, row):
                domination_counts[i] += 1

        if domination_counts[i] == 0:
            ranks[i] = 1

    current_rank = 1
    while sum(ranks == current_rank) > 0:
        for i in range(len(fitness_values)):
            if ranks[i] == current_rank:
                for j in dominated_solutions[i]:
                    domination_counts[j] -= 1
                    if domination_counts[j] == 0:
                        ranks[j] = current_rank + 1
        current_rank += 1

    return ranks

def crowding_distance(fitness_values, ranks):
    distances = np.zeros(len(fitness_values))
    for rank in np.unique(ranks):
        indices = np.where(ranks == rank)[0]
        if len(indices) == 0: continue
        rank_fitness_values = fitness_values[indices]
        sorted_indices = np.argsort(rank_fitness_values, axis=0)
        
        norm = np.max(rank_fitness_values, axis=0) - np.min(rank_fitness_values, axis=0)
        if np.any(norm == 0): norm[norm == 0] = 1  # Avoid division by zero

        distances[indices[sorted_indices[0, :]]] = np.inf
        distances[indices[sorted_indices[-1, :]]] = np.inf

        for i in range(1, len(indices) - 1):
            for j in range(rank_fitness_values.shape[1]):
                distances[indices[i]] += (rank_fitness_values[sorted_indices[i + 1, j], j] - rank_fitness_values[sorted_indices[i - 1, j], j]) / norm[j]

    return distances



def spea2_binary_tournament_selection(fitness, population_size):
    selected_indices = []
    for _ in range(population_size):
        candidate1 = random.randint(0, len(fitness) - 1)
        candidate2 = random.randint(0, len(fitness) - 1)
        while candidate2 == candidate1:  # Ensure different candidates
            candidate2 = random.randint(0, len(fitness) - 1)
        if fitness[candidate1] <= fitness[candidate2]:
            selected_indices.append(candidate1)
        else:
            selected_indices.append(candidate2)
    return selected_indices

def spea2_crossover(selected_population):
    num_parents = len(selected_population)
    num_offspring = len(selected_population)
    offspring = []
    
    for i in range(num_offspring):
        parent1 = selected_population[i % num_parents]
        parent2 = selected_population[(i + 1) % num_parents]
        crossover_point = random.randint(1, len(parent1) - 1)
        child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
        offspring.append(child)
    
    return offspring

def spea2_mutation(offspring, mutation_rate=0.1):
    mutated_offspring = []
    for individual in offspring:
        mutated_individual = individual.copy()
        for i in range(len(mutated_individual)):
            if random.random() < mutation_rate:
                mutated_individual[i] = 1 - mutated_individual[i]  # Flip the bit
        mutated_offspring.append(mutated_individual)
    return mutated_offspring

def replace_worst_individuals(population, fitness, mutated_offspring):
    # Combine original population and mutated offspring
    combined_population = np.vstack((population, mutated_offspring))

    # Sort combined population by fitness in ascending order
    sorted_indices = np.argsort(fitness)
    sorted_population = combined_population[sorted_indices]

    # Replace the worst individuals in the original population with the mutated offspring
    population[:len(mutated_offspring)] = sorted_population[:len(mutated_offspring)]

    return population

# Main Execution
if __name__ == "__main__":
    # Load or define your data and target here
    data = df.drop("type", axis=1)
    target = df["type"]
    
    # Normalize the data using MinMaxScaler
    scaler = MinMaxScaler()
    data_normalized = scaler.fit_transform(data)
    data_normalized = pd.DataFrame(data_normalized, columns=data.columns)

    # Splitting the normalized data
    data_train, data_val, target_train, target_val = train_test_split(
        data_normalized, target, test_size=0.2, random_state=42
    )

    # Setting up parameters for optimization
    population_size = 500
    archive_size = 300
    num_features = data_train.shape[1]
    max_iter = 20
    complexity_weight = 0.3  # Adjust as needed

    # Initialize metrics monitoring
    accuracy_history = []
    complexity_history = []
    
    # Initialize population and archive for feature selection algorithms
    population = np.random.randint(0, 2, (population_size, num_features))
    archive = None  # Initialize an empty archive
    
    # Initialize variables to store the best results
    best_accuracy = 0
    best_robustness = float('inf')  # Assuming lower robustness is better
    best_complexity = float('inf')  # Assuming lower complexity is better
    best_weighted_complexity_score = float('inf')
    best_features = []
    best_interpretability = float('inf') 

    # Initialize early stopping parameters
    delta = 0.001  # Minimum change in the monitored quantity to qualify as an improvement
    patience = 5  # Number of iterations with no improvement before stopping
    patience_counter = 0
  
    # Optimization loop
    for current_iter in tqdm(range(max_iter), desc="Optimizing Hybrid_SPEA2GWOABC", unit="iteration"):
        # Grey Wolf Optimizer (GWO) Phase
        gwo_feature_evaluation(population, data_train, target_train, current_iter, max_iter)

        # SPEA2 Fitness Assignment
        fitness = spea2_fitness_assignment(population, data_train, target_train)

        # SPEA2 Environmental Selection
        if archive is None:
            archive = spea2_environmental_selection(population, fitness, archive_size)
        else:
            archive = update_archive(archive, population, fitness, archive_size)

        # SPEA2 Binary Tournament Selection
        selected_indices = spea2_binary_tournament_selection(fitness, population_size)
        selected_population = population[selected_indices]

        # SPEA2 Crossover
        offspring = spea2_crossover(selected_population)

        # SPEA2 Mutation
        mutated_offspring = spea2_mutation(offspring)

        # Replace the worst individuals in the population with the mutated offspring
        population = replace_worst_individuals(population, fitness, mutated_offspring)

        # Artificial Bee Colony (ABC) Phase
        abc_feature_selection(archive, data_train, target_train)

        # Evaluate on the full archive to find the best features
        accuracy, robustness, complexity, interpretability = feature_evaluation(data_train.columns[np.argsort(np.sum(archive, axis=0))], data_val, target_val)
        best_features_from_archive = data_train.columns[np.argsort(np.sum(archive, axis=0))][-100:]  # Select top 100 features from the archive

        # Evaluate on the subset of the validation set to monitor performance
        accuracy, robustness, complexity, interpretability = feature_evaluation(best_features_from_archive, data_val, target_val)

        # Check for improvement in any metric
        is_improved = False
        if accuracy > best_accuracy + delta:
            best_accuracy = accuracy
            is_improved = True
        if robustness < best_robustness and robustness != float('inf'):  # Ensure robustness is meaningful
            best_robustness = robustness
            is_improved = True
        if complexity < best_complexity and complexity != float('inf'):  # Ensure complexity is meaningful
            best_complexity = complexity
            is_improved = True
        if complexity_weight * complexity < best_weighted_complexity_score and complexity != float('inf'):  # Ensure weighted complexity is meaningful
            best_weighted_complexity_score = complexity_weight * complexity
            is_improved = True

        # If any metric is improved, update the best features
        if is_improved:
            best_features = best_features_from_archive.copy()
            patience_counter = 0
        else:
            patience_counter += 1

        # Early stopping condition
        if patience_counter > patience:
            print("Early stopping triggered")
            break

        # Update best interpretability score
        if complexity == float('inf'):
            interpretability = float('inf')  # Set interpretability to a large finite value if no features are selected
        if interpretability < best_interpretability:
            best_interpretability = interpretability
        
        # Append metrics to history for plotting
        accuracy_history.append(accuracy)
        complexity_history.append(complexity)
   
    # Adjust the range to match the length of the accuracy_history
    iterations_completed = len(accuracy_history)

    plt.figure(figsize=(12, 6))

    # Plot for Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(range(iterations_completed), accuracy_history, label='Accuracy')
    plt.xlabel('Iteration')
    plt.ylabel('Accuracy')
    plt.title('Accuracy over Iterations')
    plt.legend()

    # Plot for Complexity (adjust similarly if complexity_history is used)
    # Ensure complexity_history is updated in each iteration
    if len(complexity_history) == iterations_completed:
        plt.subplot(1, 2, 2)
        plt.plot(range(iterations_completed), complexity_history, label='Complexity')
        plt.xlabel('Iteration')
        plt.ylabel('Complexity')
        plt.title('Complexity over Iterations')
        plt.legend()

    plt.tight_layout()
    plt.show()

    # Extracting top features after the optimization loop
    top_n = 100  # Define how many top features you want to consider
    feature_counts = np.sum(archive, axis=0)
    top_features_indices = np.argsort(feature_counts)[-top_n:]
    top_features = data_train.columns[top_features_indices]

    # Evaluate the performance on the validation set using the selected top features
    accuracy, robustness, complexity, interpretability = feature_evaluation(top_features, data_val, target_val)
    
    # Update best scores and features if current ones are better
    # Inside the optimization loop
    if accuracy > best_accuracy + delta:
        best_accuracy = accuracy
        best_robustness = robustness
        best_complexity = complexity
        best_interpretability = interpretability
        best_features = subset_top_features.copy()  # Update the best features
        patience_counter = 0  # Reset counter
    else:
        patience_counter += 1  # Increment counter

    print(f"##############################################################################")
    # Print the best results after the optimization loop
    print("\nBest Results:")
    print(f"Best Accuracy: {best_accuracy}")
    print(f"Best Robustness: {best_robustness}")
    print(f"Best Complexity: {best_complexity}")
    print(f"Best Weighted Complexity Score: {best_weighted_complexity_score}")

    print(f"##############################################################################")
    
    # Display best results after optimization loop
    print(f"Best Validation Accuracy with top {top_n} features: {best_accuracy:.4f}")
    print(f"Best Validation Robustness (Std Dev of CV scores): {best_robustness:.4f}")
    print(f"Best Complexity (1/Number of Features): {best_complexity:.4f}")
    print(f"Best Interpretability (Same as Complexity here): {best_interpretability:.4f}")
    
    print(f"##############################################################################")
    # Print the selected top features for the best result
    # End of optimization loop
    if not best_features.empty:
        print("\nSelected top features for the best result:")
        for feature in best_features:
            print(feature)
    else:
        print("No best features were selected.")

    print(f"##############################################################################")    
        
    top_features = data_train.columns[:100]  
    clf = NearestCentroid()  
    clf.fit(data_train[top_features], target_train)

    # Predict on the validation set
    predictions = clf.predict(data_val[top_features])

    # Calculate the confusion matrix
    conf_matrix = confusion_matrix(target_val, predictions)
    print("Confusion Matrix:")
    print(conf_matrix)
    plt.figure(figsize=(10, 7))
    sns.heatmap(conf_matrix, annot=True, fmt='g', cmap='viridis')
    plt.title('Confusion Matrix')
    plt.ylabel('Actual Label')
    plt.xlabel('Predicted Label')
    plt.show()
    print(f"##############################################################################")

    # Calculate classification report for more metrics
    class_report = classification_report(target_val, predictions)
    print("\nClassification Report:")
    print(class_report)
    print(f"##############################################################################")    
    # Sensitivity Analysis with Selected Top Features
    sensitivity_results = {}
    for feature in top_features:
        temp_features = [f for f in top_features if f != feature]  # Exclude the current feature
        clf.fit(data_train[temp_features], target_train)
        temp_predictions = clf.predict(data_val[temp_features])
        temp_accuracy = np.mean(temp_predictions == target_val)
        sensitivity_results[feature] = temp_accuracy
    print(f"##############################################################################")
    # Print the sensitivity analysis results
    print("\nSensitivity Analysis (Impact of leaving out each feature):")
    for feature, accuracy in sensitivity_results.items():
        print(f"{feature}: {accuracy:.4f}")
    
    #  'sensitivity_results' is a dictionary
    features = list(sensitivity_results.keys())
    accuracies = list(sensitivity_results.values())

    plt.figure(figsize=(15, 7))
    plt.bar(features, accuracies, color='skyblue')
    plt.xlabel('Features')
    plt.ylabel('Accuracy')
    plt.title('Sensitivity Analysis')
    plt.xticks(rotation=90)  # Rotates the feature names for better readability
    plt.show()
